Questions:
How to combine f string with jsonify on the same return
How to use an html input as variable


In [1]:
import pandas as pd

In [2]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, and_, or_, distinct, desc
from datetime import datetime, timedelta
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['measurement', 'station']

In [3]:
Measurement = Base.classes.measurement
Station = Base.classes.station

In [4]:
session = Session(engine)

In [5]:
inspector = inspect(engine)
for table_name in inspector.get_table_names():
    print(f'Table: {table_name}')
    print("----------")
    for column in inspector.get_columns(table_name):
        print(column['name'])

Table: measurement
----------
id
station
date
prcp
tobs
Table: station
----------
id
station
name
latitude
longitude
elevation


In [6]:
prec_df = pd.read_sql_query("SELECT * FROM measurement", engine)
prec_df.head()

,id,station,date,prcp,tobs
0,1,USC00519397,2010-01-01,0.08,65.0
1,2,USC00519397,2010-01-02,0.00,63.0
2,3,USC00519397,2010-01-03,0.00,74.0
3,4,USC00519397,2010-01-04,0.00,76.0
4,5,USC00519397,2010-01-06,NaN,73.0


In [7]:
prec_df["date"].value_counts()

2012-04-05    9
2012-11-15    9
2011-05-16    9
2010-06-01    9
2014-02-26    9
             ..
2017-08-09    3
2015-07-03    3
2017-08-22    3
2015-07-04    3
2017-08-05    3
Name: date, Length: 2792, dtype: int64

In [8]:
values = session.query(Measurement).filter(Measurement.date >= "2016-08-23").group_by(Measurement.date).all()
dict = {"Date":"Precipitation"}
for x in values:
    dict[f"{x.date}"] = x.prcp

In [9]:
stations = session.query(Station).all()
station_list = []
for x in stations:
    station_list.append(x.station)

In [10]:
station_list

['USC00519397',
 'USC00513117',
 'USC00514830',
 'USC00517948',
 'USC00518838',
 'USC00519523',
 'USC00519281',
 'USC00511918',
 'USC00516128']

In [11]:
active_station = session.query(Measurement.station, func.count(Measurement.station).label("tobs")).group_by(Measurement.station).order_by(desc("tobs")).first()[0]
active_station

'USC00519281'

In [12]:
temp_obs = session.query(Measurement).filter(and_(Measurement.station == active_station, Measurement.date >= "2016-08-23")).all()
temp_list = []
for temp in temp_obs:
    temp_list.append({temp.date, temp.tobs})

In [37]:
date_provided = input("Please type the date in a format YYYY-mm-dd: ")
minimum = session.query(func.min(Measurement.tobs).filter(Measurement.date >= date_provided)).all()[0][0]
maximum = session.query(func.max(Measurement.tobs).filter(Measurement.date >= date_provided)).all()[0][0]
average = session.query(func.avg(Measurement.tobs).filter(Measurement.date >= date_provided)).all()[0][0]
list_result = [minimum,maximum,average]

Please type the date in a format YYYY-mm-dd: 2016-08-23


In [40]:
average

74.59058295964125

In [34]:
start_date =  input("Please type the starting date in a format YYYY-mm-dd: ")
end_date =  input("Please type the end date in a format YYYY-mm-dd: ")
min_date = session.query(func.min(Measurement.tobs).filter(and_(Measurement.date >= start_date, Measurement.date <= end_date))).all()


Please type the starting date in a format YYYY-mm-dd: 08/10/2017
Please type the end date in a format YYYY-mm-dd: 2017


In [18]:
min_date = session.query(func.min(Measurement.tobs).filter(and_(Measurement.date >= start_date, Measurement.date <= end_date))).all()[0][0]
# max_date = session.query(func.max(Measurement.tobs).filter(and_(Measurement.date >= start_date, Measurement.date <= end_date)).all()[0]
# avg_date = session.query(func.avg(Measurement.tobs).filter(and_(Measurement.date >= start_date, Measurement.date <= end_date)).all()[0]
# list_result2 = [min_date, max_date, avg_date]

In [19]:
min_date

70.0